In [2]:
import numpy as np
import pandas as pd
import datetime
import re
from IPython.display import display, HTML

In [3]:
aircraft = pd.read_csv('aircraft.csv', on_bad_lines = 'skip', delimiter=';')
airlines = pd.read_csv('airlines.csv', on_bad_lines = 'skip', delimiter=',')
airports = pd.read_csv('airports.csv', on_bad_lines = 'skip', delimiter=',')
cargo_tr = pd.read_csv('cargo transportation.csv', on_bad_lines = 'skip', delimiter=';')
passenger_tr = pd.read_csv('passenger transportation.csv', on_bad_lines = 'skip', delimiter=';')

In [4]:
airlines.columns = ['№ рейса', 'Сокращенное наименование', 'Полное наименование', 'Аэропорт', 'Тип (наименование) воздушного судна']

In [5]:
airports.columns = ['Наименование аэропорта РФ', 'Документ о регистрации', 'Владелец аэропорта', 'Класс аэропорта по длине ВПП']
f_airports = airports[airports['Наименование аэропорта РФ'].notnull()]
f_airports = f_airports.reset_index(drop=True)

# Задание 1. Список уникальных типов самолетов, зарегистрированных в России. 

In [11]:
aircraft_airplane = aircraft[aircraft['Вид воздушного судна'] == 'самолет']
unique_list = aircraft_airplane['Тип (наименование) воздушного судна'].unique()
df = pd.DataFrame(unique_list) 
df.columns = ['Уникальный тип (наименование) воздушного судна']
df

,Уникальный тип (наименование) воздушного судна
0,Птенец-2
1,С-2 Елочка
2,Небесный
3,Светлый
4,Сура
...,...
1848,Ил-96-300
1849,Ил-96-300ПУ
1850,Ил-96-300ПУ(М1)
1851,Ил-96-400Т/IL96-400T


# Задание 2. Тип самолета, имеющий самую раннюю дату выдачи сертификата.

In [930]:
aircraft['дата действующего свидетельства о регистрации'] = pd.to_datetime(aircraft['дата действующего свидетельства о регистрации'], format='%d.%m.%Y')
min_data = aircraft['дата действующего свидетельства о регистрации'].min()
aircraft[aircraft['дата действующего свидетельства о регистрации'] == min_data]

,Вид воздушного судна,Тип (наименование) воздушного судна,Опознавательные знаки,Серийный №,Идентификац. №,номер свидетельства о регистрации,дата действующего свидетельства о регистрации
6635,самолет,Ан-2,RA-28885,160619,NaN,28514,1961-07-14


# Задание 3. Постройте запрос: Владелец аэропорта, Аэропорт, Суммарный пассажиропоток (2018), Суммарный грузопоток (2018). 

In [931]:
for col_name in passenger_tr.columns:
    passenger_tr.loc[(passenger_tr[col_name] == '***'), col_name] = 0

In [932]:
for col_name in cargo_tr.columns:
    cargo_tr.loc[(cargo_tr[col_name] == '***'), col_name] = 0

In [933]:
for column in cargo_tr.columns:
    if column != 'Наименование аэропорта РФ':
        a = []
        for i in range(len(cargo_tr)):
            if isinstance(cargo_tr.loc[i, column], str):
                a.append(float(cargo_tr.loc[i, column].replace(',', '.').replace(' ', '')))
            else:
                a.append(cargo_tr.loc[i, column])
        cargo_tr[column] = a

In [934]:
for column in passenger_tr.columns:
    if column != 'Наименование аэропорта РФ':
        a = []
        for i in range(len(passenger_tr)):
            if isinstance(passenger_tr.loc[i, column], str):
                a.append(float(passenger_tr.loc[i, column].replace(',', '.').replace(' ', '')))
            else:
                a.append(passenger_tr.loc[i, column])
        passenger_tr[column] = a

In [935]:
pas_cargo = passenger_tr.join(cargo_tr.set_index('Наименование аэропорта РФ'), on='Наименование аэропорта РФ', rsuffix='_cargo', lsuffix='_pas')

In [936]:
b = []
for i in range(len(f_airports['Наименование аэропорта РФ'])):
    b.append(f_airports['Наименование аэропорта РФ'][i].replace('-', ''))
f_airports['Наименование аэропорта РФ'] = b
f_airports.loc[(f_airports['Наименование аэропорта РФ'] == '  Сургут  '), 'Наименование аэропорта РФ'] = 'Сургут'

In [937]:
new_column = []
for i in range(len(f_airports['Наименование аэропорта РФ'])):
    result = re.findall(r'^\w+', f_airports['Наименование аэропорта РФ'][i])
    new_column.append(result[0])
f_airports['Наименование аэропорта РФ'] = new_column

In [938]:
с = []
for i in range(len(cargo_tr['Наименование аэропорта РФ'])):
    с.append(cargo_tr['Наименование аэропорта РФ'][i].replace('-', ''))
cargo_tr['Наименование аэропорта РФ'] = с

In [939]:
d = []
for i in range(len(passenger_tr['Наименование аэропорта РФ'])):
    d.append(passenger_tr['Наименование аэропорта РФ'][i].replace('-', ''))
passenger_tr['Наименование аэропорта РФ'] = d

In [940]:
air_pas_cargo = f_airports.join(pas_cargo.set_index('Наименование аэропорта РФ'), on='Наименование аэропорта РФ')

In [941]:
air_pas_cargo = air_pas_cargo[air_pas_cargo['Год периода данных_pas'].notnull()]
air_pas_cargo = air_pas_cargo.reset_index(drop=True)

In [942]:
df1 = pd.DataFrame()
sum_ps = air_pas_cargo.loc[:, 'Январь_pas':'Январь - Декабрь_pas':1].sum(axis=1)
df1 = pd.DataFrame(sum_ps)
df1.columns = ['Суммарный пассажиропоток']

In [943]:
df2 = pd.DataFrame()
sum_cargo = air_pas_cargo.loc[:, 'Январь_cargo':'Январь - Декабрь_cargo':1].sum(axis=1)
df2 = pd.DataFrame(sum_cargo)
df2.columns = ['Суммарный грузопоток']

In [944]:
air_pas_cargo['Суммарный грузопоток'] = sum_cargo
air_pas_cargo['Суммарный пассажиропоток'] = sum_ps

In [945]:
air_pas_cargo = air_pas_cargo.query('`Суммарный грузопоток` > 0 | `Суммарный пассажиропоток` > 0')
air_pas_cargo = air_pas_cargo.reset_index(drop=True)

In [946]:
new_air_pas_cargo = air_pas_cargo[['Наименование аэропорта РФ','Владелец аэропорта', 'Год периода данных_pas',
                                 'Год периода данных_cargo', 'Суммарный грузопоток', 'Суммарный пассажиропоток']].copy()

In [947]:
data2018 = new_air_pas_cargo.query('`Год периода данных_pas` == 2018.0 & `Год периода данных_cargo` == 2018.0')
data2018

,Наименование аэропорта РФ,Владелец аэропорта,Год периода данных_pas,Год периода данных_cargo,Суммарный грузопоток,Суммарный пассажиропоток
0,Мурманск,ОАО «Аэропорт Мурманск»,2018.0,2018.0,1287.50,713392.0
144,Сыктывкар,ОАО «Комиавиатранс»,2018.0,2018.0,469.08,446783.0
288,Воркута,ОАО «Комиавиатранс»,2018.0,2018.0,64.10,17627.0
432,Иркутск,ОАО «Международный Аэропорт Иркутск»,2018.0,2018.0,6294.32,1647187.0
576,Белоярский,ОАО «Аэропорт Белоярский»,2018.0,2018.0,63.50,50291.0
720,Когалым,ООО «Международный аэропорт Когалым»,2018.0,2018.0,177.72,104388.0
864,Надым,ОАО «Надымское авиапредприятие»,2018.0,2018.0,266.75,138880.0
1008,Нижневартовск,ОАО «Нижневартовскавиа»,2018.0,2018.0,2223.20,495859.0
1152,Ноябрьск,ОАО «Аэропорт Сургут»,2018.0,2018.0,435.05,146046.0
1296,Сургут,ОАО «Аэропорт Сургут»,2018.0,2018.0,2458.01,1358938.0


# Задание 4. Аэропорты с пассажиропотоком меньше медианы и грузопотоком больше медианы.

In [948]:
median_cargo = new_air_pas_cargo['Суммарный грузопоток'].median()

In [949]:
median_pas = new_air_pas_cargo['Суммарный пассажиропоток'].median()

In [950]:
data_median = new_air_pas_cargo.query(f'`Суммарный пассажиропоток` < {median_pas} & `Суммарный грузопоток` > {median_cargo}')

In [951]:
data_median = data_median.query('`Год периода данных_pas` == `Год периода данных_cargo`')

In [952]:
data_median

,Наименование аэропорта РФ,Владелец аэропорта,Год периода данных_pas,Год периода данных_cargo,Суммарный грузопоток,Суммарный пассажиропоток
706,Белоярский,ОАО «Аэропорт Белоярский»,2008.0,2008.0,2337.00,216916.0
719,Белоярский,ОАО «Аэропорт Белоярский»,2007.0,2007.0,2738.20,231212.0
1623,Ленск,"Мирнинское авиационное предприятие АК АЛРОСА"" ...",2015.0,2015.0,2310.00,135540.0
1662,Ленск,"Мирнинское авиационное предприятие АК АЛРОСА"" ...",2012.0,2012.0,3876.30,150312.0
1727,Ленск,"Мирнинское авиационное предприятие АК АЛРОСА"" ...",2007.0,2007.0,3086.40,103854.0
1728,Мирный,"Мирнинское авиационное предприятие АК АЛРОСА"" ...",2018.0,2018.0,5795.61,259115.0
1924,Полярный,"Мирнинское авиационное предприятие АК АЛРОСА"" ...",2014.0,2014.0,2273.24,107178.0
1937,Полярный,"Мирнинское авиационное предприятие АК АЛРОСА"" ...",2013.0,2013.0,3015.26,103706.0
1950,Полярный,"Мирнинское авиационное предприятие АК АЛРОСА"" ...",2012.0,2012.0,5120.08,103594.0
1976,Полярный,"Мирнинское авиационное предприятие АК АЛРОСА"" ...",2010.0,2010.0,2901.00,102582.0


# Задание 5. Авиакомпании, у которых нет типов воздушных судов, зарегистрированных в России.

In [953]:
lst = []
for i in range(len(airlines['Тип (наименование) воздушного судна'])):
    lst.append(list(str(airlines['Тип (наименование) воздушного судна'][i]).split(', ')))
airlines['Тип (наименование) воздушного судна'] = lst

In [954]:
new_airlines = airlines.explode('Тип (наименование) воздушного судна')

In [955]:
new_airlines['Тип (наименование) воздушного судна'] = new_airlines['Тип (наименование) воздушного судна'].apply(lambda x: x.split(' (')[0])

In [956]:
new_airlines = new_airlines.reset_index(drop=True)

In [957]:
def func(x):
    x = str(x)
    return x[:-1]

In [958]:
new_airlines['Тип (наименование) воздушного судна'] = new_airlines['Тип (наименование) воздушного судна'].apply(func)

In [959]:
f_new_airlines = new_airlines.loc[~new_airlines['Тип (наименование) воздушного судна'].isin(aircraft['Тип (наименование) воздушного судна'])]

In [960]:
data_abroad_aircraft = f_new_airlines.drop_duplicates(subset=['Сокращенное наименование'])
data_abroad_aircraft = data_abroad_aircraft.reset_index(drop=True)

In [961]:
data_rus_aircraft = new_airlines.merge(aircraft, on = 'Тип (наименование) воздушного судна', how='inner')

In [962]:
data_rus_aircraft = data_rus_aircraft.drop_duplicates(subset=['Сокращенное наименование'])
data_rus_aircraft = data_rus_aircraft.reset_index(drop=True)

In [963]:
data_only_abroad_aircraft = data_abroad_aircraft.loc[~data_abroad_aircraft['Сокращенное наименование'].isin(data_rus_aircraft['Сокращенное наименование'])]

In [964]:
data_only_abroad_aircraft = data_only_abroad_aircraft.reset_index(drop=True)
data_only_abroad_aircraft.loc[:, 'Сокращенное наименование':'Сокращенное наименование']

,Сокращенное наименование
0,Авиа Менеджмент Груп
1,Аврора
2,Ай Флай
3,АТРАН
4,Аэролимузин
5,Аэрофлот ОАО
6,ВИМ-АВИА
7,Джет Эйр Групп
8,ДОНАВИА
9,Икар ООО


# Задание 6. Таблица: месяц, суммарный пассажиропоток за данный месяц, аэропорт с максимальным пассажиропотоком.

In [965]:
new_passenger_tr = passenger_tr.sort_values(['Наименование аэропорта РФ', 'Год периода данных'])
new_passenger_tr = new_passenger_tr.reset_index(drop=True)

In [966]:
res = new_passenger_tr.groupby('Наименование аэропорта РФ', as_index=False).sum()

In [967]:
res.dropna()

,Наименование аэропорта РФ,Год периода данных,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Январь - Декабрь
0,Абакан,24150,93894.0,89402.0,99255.0,106486.0,126229.0,173358.0,201836.0,213998.0,163002.0,116850.0,104530.0,107851.0,1446158.0
1,Айхал,24150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Алдан,24150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Амдерма,24150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Анадырь,24150,39268.0,44733.0,60709.0,70880.0,85301.0,116464.0,123311.0,141765.0,102843.0,120433.0,78999.0,77030.0,982910.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,Якутск,24150,465030.0,446489.0,575052.0,632101.0,635075.0,927212.0,1110483.0,1180793.0,889951.0,663095.0,555198.0,578469.0,7951440.0
305,Якутск(Маган),24150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
306,Ямбург,24150,68655.0,65189.0,67851.0,67896.0,65643.0,63078.0,62903.0,63375.0,64520.0,60851.0,64907.0,66133.0,735173.0
307,Ярославль(Туношна),24150,7105.0,7587.0,8560.0,8101.0,8528.0,12869.0,16488.0,18962.0,19701.0,11131.0,11022.0,9973.0,121092.0


In [968]:
list_of_sum = []
for column in res.columns:
    if (column != 'Наименование аэропорта РФ') & (column !='Год периода данных'):
        list_of_sum.append((res[column]).sum())

In [969]:
dict_of_max = {}
for (columnName, columnData) in res.iteritems():
    if (columnName != 'Наименование аэропорта РФ') & (columnName !='Год периода данных'):
        dict_of_max[columnName] = max(columnData.values)

In [970]:
name_of_air = []
for key, value in dict_of_max.items():
    ind = res.index[res[key] == value].tolist()
    name_of_air.append(res['Наименование аэропорта РФ'][ind[0]])

In [971]:
moths = ['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь', 'Январь-Декабрь']

In [972]:
data = {'Месяц': moths,
        'Суммарный пассажиропоток': list_of_sum,
        'Аэропорт с максимальным пассажиропотоком': name_of_air}
max_pas = pd.DataFrame(data) 

In [973]:
max_pas

,Месяц,Суммарный пассажиропоток,Аэропорт с максимальным пассажиропотоком
0,Январь,9.258196e+07,Москва(Шереметьево)
1,Февраль,8.427920e+07,Москва(Шереметьево)
2,Март,9.754832e+07,Москва(Шереметьево)
3,Апрель,1.039993e+08,Москва(Шереметьево)
4,Май,1.207404e+08,Москва(Домодедово)
5,Июнь,1.460820e+08,Москва(Домодедово)
6,Июль,1.670433e+08,Москва(Домодедово)
7,Август,1.713780e+08,Москва(Домодедово)
8,Сентябрь,1.483724e+08,Москва(Домодедово)
9,Октябрь,1.083012e+08,Москва(Шереметьево)


# Задание 7. Таблица: Тип аэропорта, средний грузопоток в месяц в аэропортах данного типа.

In [974]:
cargo_mean = cargo_tr.groupby('Наименование аэропорта РФ', as_index=False).mean()

In [975]:
cargo_mean_other = f_airports.join(cargo_mean.set_index('Наименование аэропорта РФ'), on='Наименование аэропорта РФ')

In [976]:
def func1(x):
    return x.replace(' ', '')
cargo_mean_other['Класс аэропорта по длине ВПП'] = cargo_mean_other['Класс аэропорта по длине ВПП'].apply(func1)

In [977]:
cargo_mean_other = cargo_mean_other[cargo_mean_other['Год периода данных'].notna()].groupby('Класс аэропорта по длине ВПП', as_index=False).mean()

In [978]:
finish_cargo_mean = cargo_mean_other[['Класс аэропорта по длине ВПП', 'Январь - Декабрь']].copy()

In [979]:
finish_cargo_mean['Январь - Декабрь'] = finish_cargo_mean['Январь - Декабрь']/12

In [980]:
finish_cargo_mean.rename(columns={'Январь - Декабрь': 'Средний грузопоток в месяц в аэропортах данного типа'}, inplace=True)

In [981]:
finish_cargo_mean

,Класс аэропорта по длине ВПП,Средний грузопоток в месяц в аэропортах данного типа
0,А,462.125417
1,А-2,287.922803
2,Б,251.906705
3,Б(горный),133.399886
4,В,76.670435
5,Г,2.466292
6,Г(горный),0.000000
7,Д,0.000000
8,Е,0.000000
9,Е(горный),0.000000
